<a href="https://colab.research.google.com/github/HanhengHe/BigDataCourseProj/blob/main/bigdataproj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# GET FILE

from google.colab import drive
import zipfile

drive.mount('/content/drive/', force_remount=False)

with zipfile.ZipFile("/content/drive/MyDrive/Dataset_small/data1.zip","r") as zip_ref:
    zip_ref.extractall("targetdir")

Mounted at /content/drive/


In [34]:
# LOADING TRAINING DATA

lsTrainLabel = []
lsTrainData = []

with open("/content/targetdir/data1/train_label.txt") as f:
  for line in f.readlines():
    expression = line.split(' ')
    lsTrainLabel.append(int(expression[1]))
    dataRGB = np.load("/content/targetdir/data1/train/" + expression[0]).transpose((2,0,1))
    lsTrainData.append(dataRGB)
    # turn training data into grayscale data
    # lsData.append((dataRGB[:, :, 0] + dataRGB[:, :, 1] + dataRGB[:, :, 2]) / 3)

print("Number of traning data: {0}".format(len(lsTrainLabel)), end='\n')
print("Shape of traning data: {0}".format(lsTrainData[0].shape), end='\n')

Number of traning data: 50000
Shape of traning data: (3, 32, 32)


In [31]:
# LOAD TEST DATA

lsTestLabel = []
lsTestData = []

with open("/content/targetdir/data1/test_label.txt") as f:
  for line in f.readlines():
    expression = line.split(' ')
    lsTestLabel.append(int(expression[1]))
    dataRGB = np.load("/content/targetdir/data1/test/" + expression[0]).transpose((2,0,1))
    lsTestData.append(dataRGB)
    # turn training data into grayscale data
    # lsData.append((dataRGB[:, :, 0] + dataRGB[:, :, 1] + dataRGB[:, :, 2]) / 3)

print("Number of testing data: {0}".format(len(lsTestLabel)), end='\n')
print("Shape of testing data: {0}".format(lsTestData[0].shape), end='\n')

Number of testing data: 10000
Shape of testing data: (3, 32, 32)


In [176]:
# NETWORK DEFINE MODULE LET-NET

import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
import torch.nn.functional as F

torch.cuda._initialized = True

class CNN(nn.Module):
    def __init__(self, nClasses):
        super().__init__()

        self.cLayer1 = nn.Sequential(
          nn.Conv2d(3, 64, 5, padding=2),
          nn.BatchNorm2d(64),
          nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.cLayer2 = nn.Sequential(
          nn.Conv2d(64, 128, 5, padding=2),
          nn.BatchNorm2d(128),
          nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.cLayer3 = nn.Sequential(
          nn.Conv2d(128, 256, 5, padding=2),
          nn.BatchNorm2d(256),
          nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc = nn.Sequential(
            nn.Linear(4096, 2400),
            nn.ReLU(),
            nn.Dropout(),

            nn.Linear(2400, 740),
            nn.ReLU(),
            nn.Dropout(),

            nn.Linear(740, 74),
            nn.ReLU(),
            nn.Dropout(),

            nn.Linear(74, nClasses)
        )

      
    # forward propagation
    def forward(self, X):

        # convolutional layers
        X = self.cLayer1(X)
        X = self.cLayer2(X)
        X = self.cLayer3(X)

        # unfold
        X = torch.flatten(X, start_dim=1)

        # full connect layers
        out = self.fc(X)

        # output
        out = F.log_softmax(out, dim=-1)

        return out

In [185]:
# TRAINING MODULE CNN

import math

MINIBATCH_SIZE = 256    # mini batch size
Epochs = 100
LEARNING_RATE = 0.01
MINLOSS = 0.001
lr_diff = 1e-3
bChangeLr = True

NUMCLASS= 10
CPU = torch.device("cpu")
DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else CPU
print("DEVICE = %s" % (DEVICE))
torch.cuda.empty_cache()

trainLabels = np.array(lsTrainLabel)
trainSet = np.array(lsTrainData)

trainSet = torch.from_numpy(trainSet).float()
trainLabels = torch.from_numpy(trainLabels)
# trainLabels = torch.nn.functional.one_hot(torch.from_numpy(trainLabels), num_classes=NUMCLASS)


# first transform the data to dataset can be processed by torch
torch_dataset = Data.TensorDataset(trainSet, trainLabels.long())
# put the dataset into DataLoader
loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=MINIBATCH_SIZE,
    shuffle=True,
    num_workers=0
)

model = CNN(NUMCLASS).to(DEVICE)

nSteps = math.ceil(trainLabels.shape[0] / MINIBATCH_SIZE)
outputFrequence = 10

# optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
optimizer = optim.Adam(model.parameters())

model.train()
print("TRAINING...", end='\n')

lastLoss = 0
for epoch in range(Epochs):

    curLoss = 0

    for step, (batch_x, batch_y) in enumerate(loader):
        optimizer.zero_grad()        
        output = model(batch_x.to(DEVICE))
        loss = F.nll_loss(output, batch_y.to(DEVICE))
        loss.backward()
        optimizer.step()
        curLoss += loss.item() / nSteps
    if (epoch + 1) % outputFrequence == 0:
        if bChangeLr:
            LEARNING_RATE = LEARNING_RATE / math.pow((1 + 10 * (epoch - 1) / Epochs), 0.75)
            optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

        print(" Epoch {0}, general loss: {1}, lr = {2:.5}".format(epoch+1, curLoss, LEARNING_RATE))
    
    if curLoss <= MINLOSS:
        print(" Terminated with low loss. Epoch {0}, general loss: {1}, lr = {2:.5}".format(epoch+1, curLoss, LEARNING_RATE))
        break
    
    if abs(lastLoss - curLoss) <= lr_diff:
        print(" Terminated with unchanged loss. Epoch {0}, general loss: {1}, lr = {2:.5}".format(epoch+1, curLoss, LEARNING_RATE))
        break

    lastLoss = curLoss

# TEST MODULE

testLabels = np.array(lsTestLabel)
testSet = np.array(lsTestData)

testSet = torch.from_numpy(testSet).float()
testLabels = torch.from_numpy(testLabels).to(DEVICE)

model.eval()
print("TESTING...", end='\n')

test_loss = 0
correct = 0

# first transform the data to dataset can be processed by torch
torch_dataset = Data.TensorDataset(testSet, testLabels.long())
# put the dataset into DataLoader
loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=MINIBATCH_SIZE,
    shuffle=True,
    num_workers=0
)

with torch.no_grad():
    for step, (batch_x, batch_y) in enumerate(loader):
        output = model(batch_x.to(DEVICE))
        test_loss += F.nll_loss(output, batch_y, reduction='sum').item()  # sum a batch of loss
        predict = output.max(1, keepdim=True)[1]  # find the prediction
        correct += predict.eq(batch_y.view_as(predict)).sum().item()

testSize = testLabels.shape[0]
test_loss /= testSize
print(" Test: Average loss:%s, Accuracy: %s/%s (%s)"
      % (test_loss, correct, testSize, correct / testSize))

DEVICE = cuda:0
TRAINING...
 Epoch 10, general loss: 0.8896551089627401, lr = 0.006435
 Epoch 20, general loss: 0.6440913680256629, lr = 0.0029729
 Terminated with unchanged loss. Epoch 28, general loss: 0.5707598369644611, lr = 0.0029729
TESTING...
 Test: Average loss:0.8854738574028015, Accuracy: 7294/10000 (0.7294)


In [180]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset

class Block(nn.Module):
    
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        
    def forward(self, x):
        identity = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        x += identity
        x = self.relu(x)
        return x

class ResNet_18(nn.Module):
    
    def __init__(self, num_classes):
        
        super(ResNet_18, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        #resnet layers
        self.layer1 = self.__make_layer(64, 64, stride=1)
        self.layer2 = self.__make_layer(64, 128, stride=2)
        self.layer3 = self.__make_layer(128, 256, stride=2)
        self.layer4 = self.__make_layer(256, 512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
        
    def __make_layer(self, in_channels, out_channels, stride):
        
        identity_downsample = None
        if stride != 1:
            identity_downsample = self.identity_downsample(in_channels, out_channels)
            
        return nn.Sequential(
            Block(in_channels, out_channels, identity_downsample=identity_downsample, stride=stride), 
            Block(out_channels, out_channels)
        )
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        x = F.softmax(x, dim=1)
        return x 
    
    def identity_downsample(self, in_channels, out_channels):
        
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1), 
            nn.BatchNorm2d(out_channels)
        )

In [186]:
# TRAINING MODULE 

import math

# PARAMETERS
MINIBATCH_SIZE = 256    # mini batch size
Epochs = 100
MINLOSS = 0.001
LEARNING_RATE = 0.01
lr_diff = 1e-5
bChangeLr = True

NUMCLASS= 10
CPU = torch.device("cpu")
DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else CPU
print("DEVICE = %s" % (DEVICE))
torch.cuda.empty_cache()

trainLabels = np.array(lsTrainLabel)
trainSet = np.array(lsTrainData)

trainSet = torch.from_numpy(trainSet).float()
# trainLabels = torch.from_numpy(trainLabels)    # nll
# torch_dataset = Data.TensorDataset(trainSet, trainLabels.long())    # nll

trainLabels = torch.nn.functional.one_hot(torch.from_numpy(trainLabels), num_classes=NUMCLASS)  # mse
torch_dataset = Data.TensorDataset(trainSet, trainLabels.float())  # mse


# put the dataset into DataLoader
loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=MINIBATCH_SIZE,
    shuffle=True,
    num_workers=0
)

# model = LeNet(NUMCLASS).to(DEVICE)
model = ResNet_18(NUMCLASS).to(DEVICE)

nSteps = math.ceil(trainLabels.shape[0] / MINIBATCH_SIZE)
outputFrequence = 10

# OPTIMIZER
# optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
optimizer = optim.Adam(model.parameters())

model.train()
print("TRAINING...", end='\n')
lastLoss = 0
for epoch in range(Epochs):

    curLoss = 0

    for step, (batch_x, batch_y) in enumerate(loader):
        optimizer.zero_grad()        
        output = model(batch_x.to(DEVICE))
        # loss = F.nll_loss(output, batch_y.to(DEVICE))  # nll
        loss = F.mse_loss(output, batch_y.to(DEVICE))  # mse
        loss.backward()
        optimizer.step()
        curLoss += loss.item() / nSteps
    if (epoch + 1) % outputFrequence == 0:
        if bChangeLr:
            LEARNING_RATE = LEARNING_RATE / math.pow((1 + 10 * (epoch - 1) / Epochs), 0.75)
            optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

        print(" Epoch {0}, general loss: {1}, lr = {2:.5}".format(epoch+1, curLoss, LEARNING_RATE))

    if curLoss <= MINLOSS:
        print(" Terminated with low loss. Epoch {0}, general loss: {1}, lr = {2:.5}".format(epoch+1, curLoss, LEARNING_RATE))
        break
    
    if abs(lastLoss - curLoss) <= lr_diff:
        print(" Terminated with unchanged loss. Epoch {0}, general loss: {1}, lr = {2:.5}".format(epoch+1, curLoss, LEARNING_RATE))
        break

    lastLoss = curLoss

# TEST MODULE

testLabels = np.array(lsTestLabel)
testSet = np.array(lsTestData)

testSet = torch.from_numpy(testSet).float()
testLabels = torch.from_numpy(testLabels).to(DEVICE)

model.eval()
print("TESTING...", end='\n')

test_loss = 0
correct = 0

# first transform the data to dataset can be processed by torch
# torch_dataset = Data.TensorDataset(testSet, testLabels.long())  # nll

testLabelsOneHot = torch.nn.functional.one_hot(testLabels, num_classes=NUMCLASS)  # mse
torch_dataset = Data.TensorDataset(testSet, testLabelsOneHot.float())  # mse

# put the dataset into DataLoader
loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=MINIBATCH_SIZE,
    shuffle=False,
    num_workers=0
)

with torch.no_grad():
    for step, (batch_x, batch_y) in enumerate(loader):
        output = model(batch_x.to(DEVICE))
        # test_loss += F.nll_loss(output, batch_y, reduction='sum').item()  # nll
        test_loss += F.mse_loss(output, batch_y.to(DEVICE), reduction='sum').item()  # nll
        predict = output.max(1, keepdim=True)[1]  # find the prediction
        labels = batch_y.max(1, keepdim=True)[1]
        correct += predict.eq(labels.view_as(predict)).sum().item()

testSize = testLabels.shape[0]
test_loss /= testSize
print(" Test: Average loss:%s, Accuracy: %s/%s (%s)"
      % (test_loss, correct, testSize, correct / testSize))

DEVICE = cuda:0
TRAINING...
 Epoch 10, general loss: 0.0238948591452624, lr = 0.006435
 Epoch 20, general loss: 0.013392135885791205, lr = 0.0029729
 Epoch 30, general loss: 0.012519592400260115, lr = 0.0010923
 Terminated with unchanged loss. Epoch 32, general loss: 0.012469180073227958, lr = 0.0010923
TESTING...
 Test: Average loss:0.36894429001808166, Accuracy: 7423/10000 (0.7423)
